In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_Cancer"
cohort = "GSE178201"

# Input paths
in_trait_dir = "../../input/GEO/Liver_Cancer"
in_cohort_dir = "../../input/GEO/Liver_Cancer/GSE178201"

# Output paths
out_data_file = "../../output/preprocess/Liver_Cancer/GSE178201.csv"
out_gene_data_file = "../../output/preprocess/Liver_Cancer/gene_data/GSE178201.csv"
out_clinical_data_file = "../../output/preprocess/Liver_Cancer/clinical_data/GSE178201.csv"
json_path = "../../output/preprocess/Liver_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Dict, Any, Callable, Optional

# 1. Gene Expression Data Availability
# Based on the background information, the dataset uses the L1000 platform
# which measures the expression of ~1,000 landmark genes, so it contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Liver_Cancer):
# Looking at the tissues in the dataset, we see HEPG2 which is a hepatocellular carcinoma cell line
# However, this dataset doesn't have a cancer vs non-cancer comparison - all samples are cancer cell lines
# There's no row indicating cancer status as a variable being studied
trait_row = None  # No trait information for liver cancer comparison

# For age:
# No age information in the sample characteristics
age_row = None

# For gender:
# No gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait values to binary format."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # We would convert actual values here, but since trait_row is None, this function won't be used
    return None

def convert_age(value):
    """Convert age values to continuous format."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # We would convert actual values here, but since age_row is None, this function won't be used
    return None

def convert_gender(value):
    """Convert gender values to binary format."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # We would convert actual values here, but since gender_row is None, this function won't be used
    return None

# 3. Save Metadata
# Determine trait availability based on whether trait_row is None
is_trait_available = trait_row is not None

# Conduct initial filtering on dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we'll skip the clinical feature extraction step


### Step 3: Gene Data Extraction

In [ ]:
# Check if the dataset contains gene expression data based on previous assessment
if not is_gene_available:
    print("This dataset does not contain gene expression data (only miRNA data).")
    print("Skipping gene expression data extraction.")
else:
    # Get the matrix file directly rather than using geo_get_relevant_filepaths
    files = os.listdir(in_cohort_dir)
    if len(files) > 0:
        matrix_file = os.path.join(in_cohort_dir, files[0])
        print(f"Matrix file found: {matrix_file}")
        
        try:
            # Extract gene data
            gene_data = get_genetic_data(matrix_file)
            print(f"Gene data shape: {gene_data.shape}")
            
            # Print the first 20 gene/probe identifiers
            print("First 20 gene/probe identifiers:")
            print(gene_data.index[:20])
        except Exception as e:
            print(f"Error extracting gene data: {e}")
    else:
        print("No files found in the input directory.")


### Step 4: Gene Identifier Review

In [ ]:
# Based on the gene expression data identifiers provided (16, 23, 25, etc.),
# these appear to be numeric IDs rather than standard human gene symbols.
# Standard human gene symbols would be text-based identifiers like BRCA1, TP53, etc.
# Therefore, these identifiers will require mapping to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the appropriate columns for mapping
# Based on the gene annotation preview, 'ID' contains numeric identifiers similar to those
# in the gene expression data, and 'pr_gene_symbol' contains the gene symbols

# 2. Create gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='pr_gene_symbol')
print(f"\nGene mapping preview (first 5 rows):")
print(preview_df(gene_mapping, n=5))
print(f"Gene mapping shape: {gene_mapping.shape}")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"\nGene data after mapping (now indexed by gene symbols):")
print(f"Shape: {gene_data.shape}")
print("First 10 gene symbols:")
print(gene_data.index[:10].tolist())

# Optional: Normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nGene data after normalization:")
print(f"Shape: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10].tolist())


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the index
print("\nNormalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print("First 10 gene identifiers after normalization:")
print(normalized_gene_data.index[:10].tolist())

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Based on Step 2, we determined that trait data is not available in this dataset
print("\nNo trait data is available in this dataset for liver cancer comparison.")

# Use the clinical data that was loaded earlier instead of an empty DataFrame
# Since the dataset has clinical data (cell lines and treatments), but no trait data for our purpose
linked_data = clinical_data.copy()

# Set up variables for validation
is_gene_available = len(normalized_gene_data) > 0
is_trait_available = False  # We know this from Step 2
is_biased = True  # Since all samples are cancer cell lines, there's no control vs. cancer comparison

# 5. Conduct final quality validation and save metadata
print("\nConducting final quality validation...")
note = "This dataset contains gene expression data from human cell lines (HepG2 hepatocellular carcinoma and MCF10A breast cells) treated with different chemicals. However, it doesn't include a control vs. cancer comparison needed for liver cancer association studies."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print("Dataset deemed not usable for liver cancer associative studies due to lack of appropriate trait data.")

# Save clinical data even though it doesn't contain our target trait
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_data.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# No need to save linked_data since the dataset is not usable for our trait analysis
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved as dataset is not usable for the current trait study.")